In [1]:
! git clone https://github.com/ALjone/INF368-Final-Project

# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

fatal: destination path 'INF368-Final-Project' already exists and is not an empty directory.


In [2]:
import sys
import numpy as np
import pandas as pd
sys.path.append("/content/INF368-Final-Project")
from Bert import Bert
import os

In [3]:
data_path = "/content/INF368-Final-Project/data"
files = os.listdir(data_path)
labeled_files = [data_path+"/"+file for file in files if "train_labeled" in file]

In [4]:
batch_size = 4
seed = 0
learning_rate = 5e-5
epochs=5
results=pd.DataFrame(columns=["n_per_class", "accuracy"])


In [5]:
for n_per_class in [5,10,25,50]:
  data_file = ""
  result = {"n_per_class":n_per_class}
  bert = Bert(num_classes = 2, random_state = seed)
  for file in labeled_files:
    if f"train_labeled_{n_per_class}.csv" in file:
      data_file = file
      break
  print(data_file)
  bert.train_from_path(data_file,learning_rate=learning_rate,batch_size=batch_size,epochs=epochs)
  performance = bert.evaluate_from_path(data_path+"/test.csv")
  result["accuracy"] = performance[1]
  results = results.append(result,ignore_index=True)

    


BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3
/content/INF368-Final-Project/data/train_labeled_5.csv
Epoch 1/5
3/3 [==============================] - 21s 274ms/step - loss: 0.8665 - accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 1s 265ms/step - loss: 0.7523 - accuracy: 0.6000
Epoch 3/5
3/3 [==============================] - 1s 261ms/step - loss: 0.9101 - accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 1s 266ms/step - loss: 0.2671 - accuracy: 1.0000
Epoch 5/5
500/500 [==============================] - 20s 38ms/step - loss: 0.6965 - accuracy: 0.5600
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3
/content/INF368-Final-Project/data/train_labeled_10.csv
Epoch 1/5
5/5 [======================

In [8]:
if not os.path.exists('/content/INF368-Final-Project/results'):
      os.mkdir('/content/INF368-Final-Project/results')
results.to_csv("/content/INF368-Final-Project/results/bert_results.csv")